In [43]:
def data_cleaning(path):
    
    import pandas as pd
    import numpy as np
    import time
    
    start = time.time()
    data = pd.read_csv(path, encoding='cp949')

    # 거래금액열에 값들의 ','를 없애기
    data['거래금액(만원)'] = data['거래금액(만원)'].str.replace(',','')
    data['거래금액(만원)'] = data['거래금액(만원)'].astype(int)
    print("========== COMPLETED REPLACE")

    # data 의 시군구 컬럼에서 구이름과 동이름만 가져오기
    data['city'], data['gu'], data['dong'] = data['시군구'].str.split(' ',2).str
    print("========== COMPLETED SPLIT")
        
    # 열이름 영어이름으로 바꿔주기
    data = data.rename(columns={'단지명':'apt','전용면적(㎡)':'exclusive_area',
                                  '계약년월':'transaction_ym','계약일':'transaction_day','거래금액(만원)':'transaction_real_price',
                                  '층':'floor','건축년도':'completion_year','도로명':'addr_kr','위도':'apt_lat','경도':'apt_lng'})
    print("========== COMPLETED RENAME")
    
    ##### 거래년도,월 분리 #####
    data['transaction_y'] = (data.transaction_ym / 100).astype(int)
    data['transaction_m'] = data.transaction_ym % 100
    print("========== COMPLETED SPLIT TRANSACTION DATE")
    
    ##### 법정동코드 데이터 적용 #####
    # 법정동 코드 데이터 불러오기
    dong_code = pd.read_csv('../data/ProcessedData/dong_code.csv', encoding='cp949')
    # 법정동 코드를 동이름 기준으로 재구성
    dong_code = dong_code[['법정동코드','시군구명','읍면동명']].sort_values(['시군구명','읍면동명'])
    # 열이름 교체
    dong_code.columns = ['code','gu','dong']
    # 데이터에 법정동코드 적용
    data = pd.merge(data,dong_code, how='left')
    print("========== COMPLETED APPLY DONGCODE")
    
    ##### 지하철 데이터 적용 #####
    # 지하철 데이터 불러오기
    subway = pd.read_csv('../data/ProcessedData/subway_cnt(UI).csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,subway, how='left')
    print("========== COMPLETED APPLY SUBWAY")
    
    ##### 공원 데이터 적용  #####
    # 공원 데이터 불러오기
    park = pd.read_csv('../data/ProcessedData/park_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,park, how='left')
    # 'park_cnt' 결측치 대체
    data.park_cnt = data.park_cnt.fillna(0)
    print("========== COMPLETED APPLY PARK COUNT DATA")
    
    ##### 스타벅스 데이터 적용 #####
    # 스타벅스 데이터 불러오기
    starbucks = pd.read_csv('../data/ProcessedData/starbucks_cnt.csv', encoding='cp949')
    # 데이터 적용
    data = pd.merge(data,starbucks, how='left')
    print("========== COMPLETED APPLY STARBUCKS COUNT DATA")
    
    ##### 학원 데이터 적용 #####
    academy = pd.read_csv('../data/ProcessedData/academy_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,academy, how='left')
    print("========== COMPLETED APPLY ACADEMY COUNT DATA")
    
    ##### 어린이집 데이터 적용  #####
    # 어린이집 데이터 불러오기
    daycare = pd.read_csv('../data/ProcessedData/daycare_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,daycare, how='left')
    # 'daycare_cnt' 결측치 대체
    #data.daycare_cnt = data.daycare_cnt.fillna(0)
    print("========== COMPLETED APPLY DAY CARE CENTER COUNT DATA")
    
    ##### 유치원 데이터 전처리 #####
    kgarden = pd.read_csv('../data/ProcessedData/kindergarden_cnt.csv', encoding='cp949')
    data = pd.merge(data,kgarden, how='left')
    print("========== COMPLETED APPLY KINDERGARDEN COUNT DATA")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    #### 초,중,고,특수학교의 구별 빈도수는 어린이집,유치원,초중고(+특수)끼리는 관계가 분명하게 있지만, 아파트거래금액에는 영향이 거의 없음
    #### 그러므로 구별 초중고(+특수) 각각의 학교위치와 구별 아파트위치의 거리를 계산하여 적용 시켜보기로함
    ## 초등학교 데이터 적용 ##
    # element = pd.read_csv('data/ProcessedData/element_cnt.csv', encoding='cp949')
    # train = pd.merge(train,element, how='left')
    ## 중학교 데이터 적용 ##
    # middle = pd.read_csv('data/ProcessedData/middle_cnt.csv', encoding='cp949')
    # train = pd.merge(train,middle, how='left')
    ## 고등학교 데이터 적용 ##
    # high = pd.read_csv('data/ProcessedData/high_cnt.csv', encoding='cp949')
    # train = pd.merge(train,high, how='left')
    ## 특수학교 데이터 적용 ##
    # special = pd.read_csv('data/ProcessedData/special_cnt.csv', encoding='cp949')
    # train = pd.merge(train,special, how='left')
    # train.corr() # 아파트 거래금액과의 상관관계 계수 : -0.03,0.08,-0.01,0.01
    
    return data

In [48]:
train = data_cleaning('../data/ProcessedData/newTrain.csv')

========== COMPLETED REPLACE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  app.launch_new_instance()


========== COMPLETED SPLIT
========== COMPLETED RENAME
========== COMPLETED APPLY DONGCODE
========== COMPLETED APPLY SUBWAY
========== COMPLETED APPLY PARK COUNT DATA
========== COMPLETED APPLY STARBUCKS COUNT DATA
========== COMPLETED APPLY ACADEMY COUNT DATA
========== COMPLETED APPLY DAY CARE CENTER COUNT DATA
========== COMPLETED APPLY KINDERGARDEN COUNT DATA
---------- Done 10.05 sec ----------


In [49]:
test = data_cleaning('../data/ProcessedData/newTest.csv')

========== COMPLETED REPLACE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  app.launch_new_instance()


========== COMPLETED SPLIT
========== COMPLETED RENAME
========== COMPLETED APPLY DONGCODE
========== COMPLETED APPLY SUBWAY
========== COMPLETED APPLY PARK COUNT DATA
========== COMPLETED APPLY STARBUCKS COUNT DATA
========== COMPLETED APPLY ACADEMY COUNT DATA
========== COMPLETED APPLY DAY CARE CENTER COUNT DATA
========== COMPLETED APPLY KINDERGARDEN COUNT DATA
---------- Done 3.31 sec ----------


In [57]:
all_dt = pd.concat([train,test])
all_dt.shape

(1057565, 25)

In [58]:
duplicate = all_dt.copy()
duplicate = duplicate[['gu','dong','addr_kr','apt','apt_lat','apt_lng']].drop_duplicates()
duplicate

,gu,dong,addr_kr,apt,apt_lat,apt_lng
0,구로구,신도림동,신도림로 87,동아1,37.510960,126.882239
1,강동구,암사동,고덕로 131,롯데캐슬퍼스트,37.556344,127.141943
2,관악구,봉천동,관악로 285,봉천동아,37.487642,126.956106
3,은평구,증산동,증산서길 72,칠산(185-2),37.582730,126.906213
4,강동구,암사동,고덕로 131,롯데캐슬퍼스트,37.555167,127.143839
...,...,...,...,...,...,...
261917,강남구,대치동,삼성로51길 10-13,노빌리티빌리지,37.497295,127.060463
261942,양천구,목동,목동중앙로13나길 20,목동리버힐,37.545837,126.880034
262079,송파구,오금동,성내천로 28,우송그린,37.501212,127.141455
262474,노원구,월계동,광운로 53-17,월계로즈빌(387),37.622613,127.059155


In [59]:
def calculate_distance(data):
    
    import time
    
    func_start = time.time()
    ##### 자료 준비 #####
    import pandas as pd
    river = pd.read_csv('../data/ProcessedData/location/river_loc.csv', encoding='cp949')
    subway = pd.read_csv('../data/ProcessedData/location/subway_loc(UI).csv', encoding='cp949')
    park = pd.read_csv('../data/ProcessedData/location/park_loc.csv', encoding='cp949')
    starbucks = pd.read_csv('../data/ProcessedData/location/starbucks_loc.csv', encoding='cp949')
    '''kgarden = pd.read_csv('data/ProcessedData/location/kindergarden_loc.csv', encoding='cp949')
    element = pd.read_csv('data/ProcessedData/location/element_loc.csv', encoding='cp949')
    middle = pd.read_csv('data/ProcessedData/location/middle_loc.csv', encoding='cp949')
    high = pd.read_csv('data/ProcessedData/location/high_loc.csv', encoding='cp949')'''
    print("data is ready to calculate")
    
    ##### 거리계산 함수 #####
    from math import cos, asin, sqrt
    def distance(lat1, lon1, lat2, lon2):
        p = 0.017453292519943295     #Pi/180
        a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
        return 12742 * asin(sqrt(a)) #2*R*asin...
    print("func of distance is working.....")
    
    ##### 거리열 미리 추가 #####
    import numpy as np
    data['dist_river'] = np.nan
    data['dist_subway'] = np.nan
    data['dist_park'] = np.nan
    data['dist_starbucks'] = np.nan
    '''data['dist_kgarden'] = np.nan
    data['dist_element'] = np.nan
    data['dist_middle'] = np.nan
    data['dist_high'] = np.nan'''
    print("each of distance columns are created")
    
    start = time.time()
    # < 1 > 한강과의 최단 거리
    for i in range(len(data)):
        for j in range(len(river)):
            # 아파트 위치와 한강의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],river['lat'].iloc[j],river['lng'].iloc[j])
            if dist < data['dist_river'].iloc[i]:           # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_river'].iloc[i] = dist           # 새로 구한 거리로 대체
            elif dist > data['dist_river'].iloc[i]:         # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_river'].iloc[i] = dist               # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF 'HAN'RIVER BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 2 > 지하철과의 최단 거리
    for i in range(len(data)):
        for j in range(len(subway)):
            # 아파트 위치와 지하철 위치의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],subway['lat'].iloc[j],subway['lng'].iloc[j])
            if dist < data['dist_subway'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_subway'].iloc[i] = dist          # 새로 구한 거리로 대체
            elif dist > data['dist_subway'].iloc[i]:        # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_subway'].iloc[i] = dist              # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF SUBWAY BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    # < 3 > 공원과의 최단 거리
    start = time.time()
    for i in range(len(data)):
        for j in range(len(park)):
            # 아파트 위치와 공원의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],park['lat'].iloc[j],park['lng'].iloc[j])
            if dist < data['dist_park'].iloc[i]:            # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_park'].iloc[i] = dist            # 새로 구한 거리로 대체
            elif dist > data['dist_park'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_park'].iloc[i] = dist           # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF PARK BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 4 > 스타벅스와의 최단 거리
    for i in range(len(data)):
        for j in range(len(starbucks)):
            if data['gu'].iloc[i] == starbucks['gu'].iloc[j]:
                # 아파트 위치와 스타벅스 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],starbucks['lat'].iloc[j],starbucks['lng'].iloc[j])
                if dist < data['dist_starbucks'].iloc[i]:       # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_starbucks'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_starbucks'].iloc[i]:     # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                                    # 다른 거리를 구한다.
                data['dist_starbucks'].iloc[i] = dist           # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF STARBUCKS BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    print('---------- Done All Function {:.2f} sec ----------'.format(time.time() - func_start))
    
    return data

In [60]:
duplicate = calculate_distance(duplicate)
duplicate

data is ready to calculate
func of distance is working.....
each of distance columns are created


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


========== COMPLETED DINTANCE OF 'HAN'RIVER BETWEEN APT
---------- Done 66.46 sec ----------
========== COMPLETED DINTANCE OF SUBWAY BETWEEN APT
---------- Done 229.90 sec ----------
========== COMPLETED DINTANCE OF PARK BETWEEN APT
---------- Done 1212.59 sec ----------
========== COMPLETED DINTANCE OF STARBUCKS BETWEEN APT
---------- Done 139.49 sec ----------
---------- Done All Function 1648.49 sec ----------


,gu,dong,addr_kr,apt,apt_lat,apt_lng,dist_river,dist_subway,dist_park,dist_starbucks
0,구로구,신도림동,신도림로 87,동아1,37.510960,126.882239,3.191675,0.372833,0.548902,0.629231
1,강동구,암사동,고덕로 131,롯데캐슬퍼스트,37.556344,127.141943,1.308551,0.581991,0.027909,1.265152
2,관악구,봉천동,관악로 285,봉천동아,37.487642,126.956106,2.416219,0.770616,0.573178,0.752168
3,은평구,증산동,증산서길 72,칠산(185-2),37.582730,126.906213,2.980737,0.328170,0.042338,2.372494
4,강동구,암사동,고덕로 131,롯데캐슬퍼스트,37.555167,127.143839,1.520184,0.422415,0.239822,1.078748
...,...,...,...,...,...,...,...,...,...,...
261917,강남구,대치동,삼성로51길 10-13,노빌리티빌리지,37.497295,127.060463,2.634537,0.409423,0.181238,0.191210
261942,양천구,목동,목동중앙로13나길 20,목동리버힐,37.545837,126.880034,0.841485,0.319683,0.100515,1.030101
262079,송파구,오금동,성내천로 28,우송그린,37.501212,127.141455,4.277357,0.681203,0.113897,0.656561
262474,노원구,월계동,광운로 53-17,월계로즈빌(387),37.622613,127.059155,8.388389,0.261873,1.053638,0.150230


In [61]:
all_dt = pd.merge(all_dt,duplicate,how='left')
all_dt.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks
0,서울특별시 구로구 신도림동,643,0,동아1,84.908,201611,8,50000,1,1999,...,11,48.0,12,1281,473,38,3.191675,0.372833,0.548902,0.629231


In [62]:
def influence_area(data):
    import numpy as np
    # 역세권
    super_SIA = int(np.round(data.query("dist_subway <= 0.2").transaction_real_price.values.mean(),0))
    SIA_1st = int(np.round(data.query("dist_subway > 0.2 & dist_subway <= 0.5").transaction_real_price.values.mean(),0))
    SIA_2nd = int(np.round(data.query("dist_subway > 0.5 & dist_subway <= 1.0").transaction_real_price.values.mean(),0))
    not_SIA = int(np.round(data.query("dist_subway > 1").transaction_real_price.values.mean(),0))
    
    data.loc[data.dist_subway <= 0.2, "StationIA(mean_price)"] = super_SIA
    data.loc[(data.dist_subway > 0.2)&(data.dist_subway <= 0.5), "StationIA(mean_price)"] = SIA_1st
    data.loc[(data.dist_subway > 0.5)&(data.dist_subway <= 1), "StationIA(mean_price)"] = SIA_2nd
    data.loc[data.dist_subway > 1, "StationIA(mean_price)"] = not_SIA

    # 스(타벅스)세권
    starbucks_1st = int(np.round(data.query("dist_starbucks <= 0.2").transaction_real_price.values.mean(),0))
    starbucks_2nd = int(np.round(data.query("dist_starbucks > 0.2 & dist_starbucks <= 0.5").transaction_real_price.values.mean(),0))
    starbucks_3rd = int(np.round(data.query("dist_starbucks > 0.5 & dist_starbucks <= 1.0").transaction_real_price.values.mean(),0))
    starbucks_4th = int(np.round(data.query("dist_starbucks > 1").transaction_real_price.values.mean(),0))
    
    data.loc[data.dist_starbucks <= 0.2, "StarbucksIA(mean_price)"] = starbucks_1st
    data.loc[(data.dist_starbucks > 0.2)&(data.dist_starbucks <= 0.5), "StarbucksIA(mean_price)"] = starbucks_2nd
    data.loc[(data.dist_starbucks > 0.5)&(data.dist_starbucks <= 1), "StarbucksIA(mean_price)"] = starbucks_3rd
    data.loc[data.dist_starbucks > 1, "StarbucksIA(mean_price)"] = starbucks_4th

    # 숲세권
    park_1st = int(np.round(data.query("dist_park <= 0.2").transaction_real_price.values.mean(),0))
    park_2nd = int(np.round(data.query("dist_park > 0.2 & dist_park <= 0.5").transaction_real_price.values.mean(),0))
    park_3rd = int(np.round(data.query("dist_park > 0.5 & dist_park <= 1.0").transaction_real_price.values.mean(),0))
    park_4th = int(np.round(data.query("dist_park > 1").transaction_real_price.values.mean(),0))

    data.loc[data.dist_park <= 0.2, "ParkIA(mean_price)"] = park_1st
    data.loc[(data.dist_park > 0.2)&(data.dist_park <= 0.5), "ParkIA(mean_price)"] = park_2nd
    data.loc[(data.dist_park > 0.5)&(data.dist_park <= 1), "ParkIA(mean_price)"] = park_3rd
    data.loc[data.dist_park > 1, "ParkIA(mean_price)"] = park_4th
    
    return data

In [63]:
all_dt = influence_area(all_dt)
all_dt.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 구로구 신도림동,643,0,동아1,84.908,201611,8,50000,1,1999,...,1281,473,38,3.191675,0.372833,0.548902,0.629231,52499.0,46844.0,44941.0


In [64]:
all_dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057565 entries, 0 to 1057564
Data columns (total 32 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   시군구                      1057565 non-null  object 
 1   본번                       1057565 non-null  int64  
 2   부번                       1057565 non-null  int64  
 3   apt                      1057565 non-null  object 
 4   exclusive_area           1057565 non-null  float64
 5   transaction_ym           1057565 non-null  int64  
 6   transaction_day          1057565 non-null  int64  
 7   transaction_real_price   1057565 non-null  int64  
 8   floor                    1057565 non-null  int64  
 9   completion_year          1057565 non-null  int64  
 10  addr_kr                  1057565 non-null  object 
 11  apt_lat                  1057565 non-null  float64
 12  apt_lng                  1057565 non-null  float64
 13  city                     1057565 non-null 

In [65]:
all_dt.to_csv('processedData.csv', index=False, encoding='cp949')

In [69]:
all_dt = pd.read_csv('processedData.csv', encoding='cp949')
all_dt.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 구로구 신도림동,643,0,동아1,84.908,201611,8,50000,1,1999,...,1281,473,38,3.191675,0.372833,0.548902,0.629231,52499.0,46844.0,44941.0


In [71]:
work = pd.read_csv('서울지역고용통계.csv', encoding='cp949', header)
work

,기간,자치구,동,합계,합계.1,합계.2,농업 임업 및 어업,농업 임업 및 어업.1,광업,광업.1,...,공공행정 국방 및 사회보장 행정,공공행정 국방 및 사회보장 행정.1,교육 서비스업,교육 서비스업.1,보건업 및 사회복지 서비스업,보건업 및 사회복지 서비스업.1,예술 스포츠 및 여가관련 서비스업,예술 스포츠 및 여가관련 서비스업.1,협회 및 단체 수리 및 기타 개인 서비스업,협회 및 단체 수리 및 기타 개인 서비스업.1
0,기간,자치구,동,사업체수,총종사자수,여성종사자수,계,여성종사자수,계,여성종사자수,...,계,여성종사자수,계,여성종사자수,계,여성종사자수,계,여성종사자수,계,여성종사자수
1,2018,합계,합계,"823,385","5,210,936","2,332,709",462,78,81,19,...,"140,121","52,738","346,219","211,444","378,366","299,776","91,104","42,165","191,423","97,409"
2,2018,종로구,소계,"39,952","265,017","112,300",17,4,10,2,...,"15,658","5,839","13,504","7,317","15,888","10,685","6,227","2,983","7,672","3,680"
3,2018,종로구,사직동,"3,541","49,536","20,878",14,3,-,-,...,"6,312","2,795",967,614,870,673,"1,544",693,843,464
4,2018,종로구,삼청동,712,"4,577","2,384",-,-,-,-,...,"1,204",292,260,144,30,21,175,124,209,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,2018,강동구,둔촌1동,19,341,111,-,-,-,-,...,8,3,158,56,-,-,-,-,5,-
447,2018,강동구,둔촌2동,"1,601","9,718","4,918",-,-,-,-,...,69,28,575,421,"3,122","2,248",124,64,342,176
448,2018,강동구,암사1동,"1,916","5,756","2,799",-,-,-,-,...,31,21,271,192,962,824,149,83,404,223
449,2018,강동구,천호2동,"3,736","14,933","7,829",-,-,-,-,...,102,20,604,431,"1,626","1,225",487,245,877,504


In [4]:
from keras import models
from keras import layers
import keras.backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [30]:
K.clear_session()
original_model = models.Sequential()
original_model.add(layers.Dense(16, activation='relu', input_shape=(7,)))
original_model.add(layers.Dense(16, activation='relu'))
original_model.add(layers.Dense(1, activation='sigmoid'))

original_model.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['acc'])

In [31]:
smaller_model = models.Sequential()
smaller_model.add(layers.Dense(4, activation='relu', input_shape=(7,)))
smaller_model.add(layers.Dense(4, activation='relu'))
smaller_model.add(layers.Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy',
                      metrics=['acc'])

In [32]:
original_hist = original_model.fit(x_train, y_train,
                                   epochs=20,
                                   batch_size=256,
                                   validation_data=(x_test, y_test),
                                   verbose=2)

Train on 793173 samples, validate on 264392 samples
Epoch 1/20
 - 3s - loss: -1.3066e+16 - acc: 0.0000e+00 - val_loss: -4.7982e+16 - val_acc: 0.0000e+00
Epoch 2/20
 - 3s - loss: -1.6306e+17 - acc: 0.0000e+00 - val_loss: -3.3636e+17 - val_acc: 0.0000e+00
Epoch 3/20
 - 3s - loss: -6.6198e+17 - acc: 0.0000e+00 - val_loss: -1.0829e+18 - val_acc: 0.0000e+00
Epoch 4/20
 - 3s - loss: -1.7285e+18 - acc: 0.0000e+00 - val_loss: -2.5054e+18 - val_acc: 0.0000e+00
Epoch 5/20
 - 3s - loss: -3.5788e+18 - acc: 0.0000e+00 - val_loss: -4.8211e+18 - val_acc: 0.0000e+00
Epoch 6/20
 - 3s - loss: -6.4302e+18 - acc: 0.0000e+00 - val_loss: -8.2471e+18 - val_acc: 0.0000e+00
Epoch 7/20
 - 3s - loss: -1.0502e+19 - acc: 0.0000e+00 - val_loss: -1.3001e+19 - val_acc: 0.0000e+00
Epoch 8/20
 - 3s - loss: -1.6006e+19 - acc: 0.0000e+00 - val_loss: -1.9300e+19 - val_acc: 0.0000e+00
Epoch 9/20
 - 3s - loss: -2.3166e+19 - acc: 0.0000e+00 - val_loss: -2.7360e+19 - val_acc: 0.0000e+00
Epoch 10/20
 - 3s - loss: -3.2193e+19 -

In [33]:
smaller_model_hist = smaller_model.fit(x_train, y_train,
                                       epochs=20,
                                       batch_size=256,
                                       validation_data=(x_test, y_test),
                                       verbose=1)

Train on 793173 samples, validate on 264392 samples
Epoch 1/20
793173/793173 [==============================] - 3s 4us/step - loss: -4104565551569437.0000 - acc: 0.0000e+00 - val_loss: -14744590099406480.0000 - val_acc: 0.0000e+00
Epoch 2/20
793173/793173 [==============================] - 3s 4us/step - loss: -48849791838489376.0000 - acc: 0.0000e+00 - val_loss: -99920778123951008.0000 - val_acc: 0.0000e+00
Epoch 3/20
793173/793173 [==============================] - 3s 4us/step - loss: -195123403282012224.0000 - acc: 0.0000e+00 - val_loss: -317791810478585152.0000 - val_acc: 0.0000e+00
Epoch 4/20
793173/793173 [==============================] - 4s 4us/step - loss: -504973241583356096.0000 - acc: 0.0000e+00 - val_loss: -730525421417160448.0000 - val_acc: 0.0000e+00
Epoch 5/20
793173/793173 [==============================] - 3s 4us/step - loss: -1041074993488119808.0000 - acc: 0.0000e+00 - val_loss: -1400161300950757120.0000 - val_acc: 0.0000e+00
Epoch 6/20
793173/793173 [===============